We use are currently using Ollama, installed using the setup provided on their official website.
Finally, to run a model:
```sh
ollama run gemma3:1b

```
To self-host a small LLM Model with a size of 815MB.

To try a more powerful model:
```sh
ollama run deepseek-r1:7b
```

In [2]:
def get_review_prompt(review: str = ""):
    return ("You are an AI Model that specializes in assessing mobile application reviews.\n"
            + "You must identify only the negative experiences and return them.\n"
            + "You must only return issues that are due to the application software."
            + 'You must respond with exactly a string array containing 0 or many of the items found here: ["Functional Suitability", "Performance Efficiency", "Compatibility", "Interaction Capability", "Reliability", "Security", "Maintainability", "Flexibility", "Safety"]'
            + 'Functional Suitability: This characteristic represents the degree to which a product or system provides functions that meet stated and implied needs when used under specified conditions.'
            + 'Performance Efficiency: This characteristic represents the degree to which a product performs its functions within specified time and throughput parameters and is efficient in the use of resources (such as CPU, memory, storage, network devices, energy, materials...) under specified conditions.'
            +' Compatibility: Degree to which a product, system or component can exchange information with other products, systems or components, and/or perform its required functions while sharing the same common environment and resources.'
            +' Interaction Capability: Degree to which a product or system can be interacted with by specified users to exchange information ia the user interface to complete specific tasks in a variety of contexts of use.'
            + 'Reliability: Degree to which a system, product or component performs specified functions under specified conditions for a specified period of time.'
            +'Security: Degree to which a product or system defends against attack patterns by malicious actors and protects information and data so that persons or other products or systems have the degree of data access appropriate to their types and levels of authorization.'
            +'Maintainability: This characteristic represents the degree of effectiveness and efficiency with which a product or system can be modified to improve it, correct it or adapt it to changes in environment, and in requirements.'
            +'Flexibility: Degree to which a product can be adapted to changes in its requirements, contexts of use or system environment.'
            + 'Safety: This characteristic represents the degree to which a product under defined conditions to avoid a state in which human life, health, property, or the environment is endangered.'
            + f'Review: "{review}"')

def extract_assessment_metrics_array(request_response):
    response_text = request_response.json()["response"]
    return response_text.split("\n")[-1]
    

In [3]:
import requests

example_review = "I truly enjoyed this app. All of a sudden, (within the last 2 weeks) when I open the app, it shuts down and I end up on my home page. I refreshed my phone, (Samsung Galaxy Note 10 Plys) keep turning it off and then turn it back on, to no avail. Please help me out."

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "deepseek-r1:7b",
          "prompt": get_review_prompt(example_review),
          "stream": False
          },
)

print(extract_assessment_metrics_array(response))

["Functional Suitability"]


In [4]:
response_text = response.json()["response"]
response_text

'<think>\nOkay, so I\'m looking at this user query where they want an AI model that assesses mobile app reviews and only returns the negative experiences related to specific software issues. The model should respond with a string array containing certain predefined items: Functional Suitability, Performance Efficiency, Compatibility, Interaction Capability, Reliability, Security, Maintainability, Flexibility, and Safety.\n\nThe user provided a sample review which is a bit informal but mentions that the app shuts down when opened, causing the user to go back to their home page after refreshing. The key negative experience here seems to be related to the app shutting down unexpectedly, which affects functionality. \n\nI need to check if this issue falls under any of the listed categories. Functional Suitability is about whether the app meets the user\'s needs as stated or implied. Since the app is malfunctioning when opened, it doesn\'t perform its intended function properly, so that def

In [5]:
# Add a column with predictions
import pandas as pd
df = pd.read_csv('Health_Apps_Review.csv')

In [6]:
df.head(5)

,appsname,review,rating
0,Samsung Health,I did like this app 'cause it keeps my daily r...,1.0
1,Samsung Health,"I love the app, but unfortunately it is draini...",3.0
2,Samsung Health,"Overall, not super impressed. It doesn't sync ...",2.0
3,Samsung Health,"I used to love this app, but it has had so man...",1.0
4,Samsung Health,Everything was going great until the app was r...,1.0


In [ ]:
i = 0
for row in df.itertuples(index=True):
    print(f"Index: {row.Index}, review: {row.review}")
    if i == 5:
        break
    i = i + 1

In [8]:
def request_and_extract_assessment_metrics(review: str):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "deepseek-r1:7b",
              "prompt": get_review_prompt(review),
              "stream": False
              },
    )

    return extract_assessment_metrics_array(response)

In [10]:
for row in df.itertuples(index=True):
    if row.Index > 198:
        df.at[row.Index, 'classification'] = request_and_extract_assessment_metrics(row.review)

In [11]:
df.to_csv('output.csv', index=False)